In [3]:
import os
import json
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)
import tensorflow_io as tfio
print(tfio.__version__)

from tensorflow.keras import layers
from tensorflow.keras import models

2023-04-23 16:19:37.342445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 16:19:49.188349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-23 16:19:49.188602: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

2.11.0
0.29.0


In [8]:
# train.csv file
df = pd.read_csv("../data/train.csv")

# Get labels 2 id
with open("../data/sign_to_prediction_index_map.json") as f:
    label2id = json.load(f)
    
def add_path(row):
    return "../data/"+row.path
    
df["sign"] = df["sign"].apply(lambda sign: label2id[sign])
df["path"] = df.apply(lambda row: add_path(row), axis=1)
df.head()

,path,participant_id,sequence_id,sign
0,../data/train_landmark_files/26734/1000035562....,26734,1000035562,25
1,../data/train_landmark_files/28656/1000106739....,28656,1000106739,232
2,../data/train_landmark_files/16069/100015657.p...,16069,100015657,48
3,../data/train_landmark_files/25571/1000210073....,25571,1000210073,23
4,../data/train_landmark_files/62590/1000240708....,62590,1000240708,164


In [15]:
for participant_id, tmp_df in df.groupby("participant_id"):
    print(participant_id)
    break

2044


In [16]:
tmp_df

,path,participant_id,sequence_id,sign
46,../data/train_landmark_files/2044/1001950812.p...,2044,1001950812,142
52,../data/train_landmark_files/2044/1002091184.p...,2044,1002091184,67
53,../data/train_landmark_files/2044/1002092995.p...,2044,1002092995,25
87,../data/train_landmark_files/2044/1003433922.p...,2044,1003433922,206
105,../data/train_landmark_files/2044/1004220951.p...,2044,1004220951,247
...,...,...,...,...
94355,../data/train_landmark_files/2044/994928291.pa...,2044,994928291,81
94395,../data/train_landmark_files/2044/996571864.pa...,2044,996571864,149
94404,../data/train_landmark_files/2044/996830210.pa...,2044,996830210,41
94438,../data/train_landmark_files/2044/998355330.pa...,2044,998355330,31


In [9]:
df.participant_id.value_counts()

49445    4968
61333    4900
36257    4896
16069    4848
26734    4841
55372    4826
2044     4810
37779    4782
32319    4753
29302    4722
22343    4677
53618    4656
37055    4648
28656    4563
62590    4563
34503    4545
27610    4275
25571    3865
18796    3502
4718     3499
30680    3338
Name: participant_id, dtype: int64

In [11]:
df.participant_id.unique()

array([26734, 28656, 16069, 25571, 62590, 32319, 37055, 29302, 49445,
       36257, 22343, 27610, 61333, 53618, 34503, 18796,  4718, 55372,
        2044, 37779, 30680])

In [10]:
paths = df.path.values
labels = df.sign.values

NUM_REPS = 543

for path, label in zip(paths, labels):
    print(path, label)
    frames_df = pd.read_parquet(path)[["x", "y", "z"]]
    print(len(frames_df))
    break

../data/train_landmark_files/26734/1000035562.parquet 25
12489


In [ ]:
tfrecords_dir = "../data/tfrecords-participants"

num_samples = 4096
num_tfrecords = len(df) // num_samples
if len(df) % num_samples:
    num_tfrecords += 1  # add one record if there are any remaining samples

if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)  # creating TFRecords output folder

In [ ]:
print(num_tfrecords)

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=num_tfrecords, random_state=None, shuffle=False)

stratified_labels = {}

for i, (_, test_index) in enumerate(skf.split(paths, labels)):
    print(f"Fold {i}:")
    print(f"  Test:  index={test_index}")
    
    stratified_labels[i] = test_index

In [ ]:
stratified_labels

In [ ]:
for k, v in stratified_labels.items():
    split_paths = paths[v]
    split_labels = labels[v]
    
    for path, label in zip(split_paths, split_labels):
        frames = pd.read_parquet(path)[["x", "y", "z"]].values.astype(np.float32)
        n_frames = len(frames)/543
        
        break
    break

In [ ]:
n_frames

In [ ]:
frames.shape

In [ ]:
label

In [ ]:
frames.reshape(int(n_frames), 543, 3)

In [ ]:
st = tf.io.serialize_tensor(frames)

In [ ]:
tf.train.Feature(bytes_list=tf.train.BytesList(value=[st.numpy()]))

In [ ]:
pt = tf.io.parse_tensor(st, out_type=tf.float32)

In [ ]:
pt.numpy()

In [ ]:
frames

In [9]:
def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_sequence(sequence):
    """Returns a list of float_list from a float / double."""
    feature_list = [tf.train.Feature(float_list=tf.train.FloatList(value=value.tolist())) for value in sequence]
    return tf.train.FeatureList(feature=feature_list)


def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.numpy()]))


def serialize_sequence(sequence):
    """Serialize the multidimentional tensor"""
    return tf.io.serialize_tensor(sequence)


def parse_sequence(serialized_sequence):
    return tf.io.parse_tensor(
        serialized_sequence,
        out_type=tf.float16,
    )


def create_example(n_frames, sequence, label):
    feature = {
        "n_frames": float_feature(n_frames),
        "frames": bytes_feature(serialize_sequence(frames)),
        "label": int64_feature(label),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))


# def create_example(n_frame, seq, label):
#     sequence_features = tf.train.FeatureLists(feature_list={"frames": float_sequence(frames)})

#     context_features = tf.train.Features(feature = {
#         "n_frames": float_feature(n_frames),
#         "label": int64_feature(label),
#     })

#     example = tf.train.SequenceExample(context=context_features, feature_lists=sequence_features)
    
#     return example

# sequence_features = {
#   "frames": tf.io.FixedLenSequenceFeature([], dtype=tf.float32)
# }

# context_features = {
#   "n_frames": tf.io.FixedLenFeature([], tf.float32),
#   "label": tf.io.FixedLenFeature([], tf.int64),
# }


# context, sequence = tf.io.parse_single_sequence_example(
#     example,
#     context_features=context_features, 
#     sequence_features=sequence_features
# )


# def parse_tfrecord_fn(example):
#     feature_description = {
#         "n_frames": tf.io.FixedLenFeature([], tf.float32),
#         "frames": tf.io.FixedLenFeature([], tf.string),
#         "label": tf.io.FixedLenFeature([], tf.int64),
#     }
#     example = tf.io.parse_single_example(example, feature_description)
    
#     n_frames = example["n_frames"]
#     label = tf.one_hot(example["label"], depth=250)
#     frames = tf.reshape(parse_sequence(example["frames"]), shape=(n_frames, 543, 3))
    
#     return example

def parse_tfrecord_fn(example):
    feature_description = {
        "n_frames": tf.io.FixedLenFeature([], tf.float32),
        "frames": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    
    return tf.io.parse_single_example(example, feature_description)

In [ ]:
example = create_example(
    n_frames,
    frames,
    label
)
example

In [ ]:
with tf.io.TFRecordWriter(
    tfrecords_dir + "sample.tfrec"
) as writer:
    example = create_example(
        n_frames,
        frames,
        label
    )
    writer.write(example.SerializeToString())

In [18]:
tfrec_path = "../data/tfrecord_heatmaps_tmp/2044_chunk_0.tfrec"
tfrec_path

'../data/tfrecord_heatmaps_tmp/2044_chunk_0.tfrec'

In [19]:
raw_dataset = tf.data.TFRecordDataset(tfrec_path)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [23]:
example = parse_tfrecord_fn(next(iter(raw_dataset)))

In [24]:
def parse_sequence(serialized_sequence):
    return tf.io.parse_tensor(
        serialized_sequence,
        out_type=tf.float16,
    )

In [25]:
frames = tf.reshape(parse_sequence(example["frames"]), shape=(28, 81, 48, 48)).numpy()

In [26]:
frames

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [27]:
def vis_heatmaps(heatmaps, channel=-1, ratio=8):
    # if channel is -1, draw all keypoints / limbs on the same map
    import matplotlib.cm as cm
    heatmaps = [x.transpose(1, 2, 0) for x in heatmaps]
    h, w, _ = heatmaps[0].shape
    newh, neww = int(h * ratio), int(w * ratio)

    if channel == -1:
        heatmaps = [np.max(x, axis=-1) for x in heatmaps]
    cmap = cm.viridis
    heatmaps = [(cmap(x)[..., :3] * 255).astype(np.uint8) for x in heatmaps]
    heatmaps = [cv2.resize(x, (neww, newh)) for x in heatmaps]
    return heatmaps

In [29]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots(figsize=(5, 8))

arr = frames

heatmap = vis_heatmaps(arr)

def update(i):
    im_normed = heatmap[i]
    ax.imshow(im_normed)
    ax.set_title(f"human_{i}", fontsize=20)
    ax.set_axis_off()


anim = FuncAnimation(fig, update, frames=np.arange(0, len(arr)), interval=10)
anim.save('humanpose3d-resized2.gif', dpi=80, writer='imagemagick')
plt.close()

MovieWriter imagemagick unavailable; using Pillow instead.


In [11]:
example["n_frames"]

<tf.Tensor: shape=(), dtype=float32, numpy=28.0>

In [ ]:
tf.one_hot(example["label"], depth=250)

In [ ]:
tf.reshape(parse_sequence(example["frames"]), shape=(example["n_frames"], 543, 3))

In [ ]:
np.isnan(parse_sequence(example["frames"]).numpy()).sum()

In [ ]:
trp = tfrecords_dir+f"/stratified_split_{1}.tfrec"

In [ ]:
raw_dataset = tf.data.TFRecordDataset(tfrec_path)
raw_dataset

In [ ]:
example = parse_tfrecord_fn(next(iter(raw_dataset)))

In [ ]:
example